In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

df=pd.read_csv('raw_with_170labels_version2.csv')
low_con_df=pd.read_csv('low_confidence_revised.csv')
high_con_df=pd.read_csv('df_high_confidence_revised.csv')

In [2]:
df

,text,label
0,First time mom here. My husband recently start...,0.0
1,I gained 80 pounds while I was pregnant I went...,1.0
2,i’m not usually one to ask for help when i nee...,0.0
3,I (31f) and my husband (34m) just had our firs...,0.0
4,"Hi there, 28F with 2nd baby. 6 weeks PP and wa...",0.0
...,...,...
952,"I hit 7 months postpartum tomorrow, and I thin...",NaN
953,PPD sucks. I’m overwhelmed by everything. Even...,NaN
954,Hello! I’m a first time c section mom who’s 7 ...,NaN
955,I finally reached breaking point today. I thin...,NaN


In [ ]:

low_con_filtered = low_con_df[['Unnamed: 1', 'text', 'Human_label']]


low_con_filtered.set_index('Unnamed: 1', inplace=True)

In [4]:
low_con_filtered

,text,Human_label
Unnamed: 1,,
170,I’m 5 months pp with my second and I’m a singl...,0
172,My SO intentionally missed work today to spend...,0
174,I’m almost 24 weeks postpartum and just really...,1
180,please don’t judge.. i’m genuinely terrified.....,1
187,I feel like I’ve totally lost myself and I am ...,2
...,...,...
931,I don’t know how to snap out of it. I’ve got a...,0
937,Idk how to write this. I feel so alone right n...,0
941,I have been struggling horribly with PPD and P...,0


In [6]:
high_con_filtered = high_con_df[['Unnamed: 0', 'text', 'human_label']]
high_con_filtered.set_index('Unnamed: 0', inplace=True)
high_con_filtered

,text,human_label
Unnamed: 0,,
171,I've been really struggling this first week an...,1.0
177,Becoz of lack of sleep at night and baby wakin...,0.0
186,The left side is me at 6 weeks pregnant and th...,1.0
189,Can anyone recommend online therapy for post p...,0.0
191,What helps everyone not get so lost in motherh...,0.0
...,...,...
939,I was prescribed it and just any side effects ...,0.0
940,Is this a good place to seek help for potentia...,0.0
945,My son is 4 months old. He started smiling som...,0.0


In [ ]:

df.loc[low_con_filtered.index, 'label'] = low_con_filtered['Human_label']

df.loc[high_con_filtered.index, 'label'] = high_con_filtered['human_label']

In [9]:
df

,text,label
0,First time mom here. My husband recently start...,0.0
1,I gained 80 pounds while I was pregnant I went...,1.0
2,i’m not usually one to ask for help when i nee...,0.0
3,I (31f) and my husband (34m) just had our firs...,0.0
4,"Hi there, 28F with 2nd baby. 6 weeks PP and wa...",0.0
...,...,...
952,"I hit 7 months postpartum tomorrow, and I thin...",0.0
953,PPD sucks. I’m overwhelmed by everything. Even...,1.0
954,Hello! I’m a first time c section mom who’s 7 ...,NaN
955,I finally reached breaking point today. I thin...,0.0


In [ ]:

df_train = df[df['label'].notna()]  
df_test = df[df['label'].isna()]   

print("训练集样本数:", df_train.shape[0])
print("测试集样本数:", df_test.shape[0])

print(df_train.head())
print(df_test.head())


训练集样本数: 441
测试集样本数: 516
                                                text  label
0  First time mom here. My husband recently start...    0.0
1  I gained 80 pounds while I was pregnant I went...    1.0
2  i’m not usually one to ask for help when i nee...    0.0
3  I (31f) and my husband (34m) just had our firs...    0.0
4  Hi there, 28F with 2nd baby. 6 weeks PP and wa...    0.0
                                                  text  label
173  Has anyone been hospitalised in a (mbu) mother...    NaN
175  I’m 4 months PP and I’m still feeling this way...    NaN
176  Hey all! A few months ago I had posted here as...    NaN
178  Few weeks ago after DCF was investigating my f...    NaN
179  I don’t know where I can vent so posting this ...    NaN


In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
import nltk
import spacy

nltk.download('stopwords')
from nltk.corpus import stopwords

nlp = spacy.load('en_core_web_sm')

df_unlabeled=df_test


def preprocess_text(text):

    text = text.lower()

    text = ''.join([char for char in text if char.isalpha() or char.isspace()])

    stop_words = set(stopwords.words('english'))
    doc = nlp(text)
    text = ' '.join([token.lemma_ for token in doc if token.text not in stop_words])

    return text

def detect_suicide_risk(text):
    suicide_keywords = ['suicide', 'kill myself', 'death', 'self-harm', 'suicidal',
                        "gas", "pesticide", "herbicide", "poison", "hang", "drown",
                        "jump", "gun", "immolation", "stab"]
    text_lower = text.lower()

    for word in suicide_keywords:
        if word in text_lower:
            return True
    return False


df_train['processed_text'] = df_train['text'].apply(preprocess_text)
df_unlabeled['processed_text'] = df_unlabeled['text'].apply(preprocess_text)


X = df_train['processed_text']
y = df_train['label']


vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))  # 增加n-grams范围
X_tfidf = vectorizer.fit_transform(X)

smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_tfidf, y)

logreg = LogisticRegression(max_iter=1000)
svm = SVC(kernel='linear', random_state=42, probability=True)  # SVM with probability enabled

param_grid_logreg = {'C': [0.1, 1, 10], 'solver': ['liblinear', 'saga']}
grid_search_logreg = GridSearchCV(logreg, param_grid_logreg, cv=5, n_jobs=-1)
grid_search_logreg.fit(X_resampled, y_resampled)
best_logreg = grid_search_logreg.best_estimator_

param_grid_svm = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}
grid_search_svm = GridSearchCV(svm, param_grid_svm, cv=5, n_jobs=-1)
grid_search_svm.fit(X_resampled, y_resampled)
best_svm = grid_search_svm.best_estimator_

voting_clf = VotingClassifier(estimators=[('logreg', best_logreg), ('svm', best_svm)], voting='soft')

voting_clf.fit(X_resampled, y_resampled)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

val_preds, val_true = [], []

for train_idx, val_idx in skf.split(X_resampled, y_resampled):
    X_train, X_val = X_resampled[train_idx], X_resampled[val_idx]
    y_train, y_val = y_resampled.iloc[train_idx], y_resampled.iloc[val_idx]

    voting_clf.fit(X_train, y_train)
    preds = voting_clf.predict(X_val)

    val_preds.extend(preds)
    val_true.extend(y_val)

print(classification_report(val_true, val_preds, digits=4))

X_unlabeled_tfidf = vectorizer.transform(df_unlabeled['processed_text'])

probs = voting_clf.predict_proba(X_unlabeled_tfidf)  # shape: [n_samples, n_classes]

confidences = probs.max(axis=1)

preds = voting_clf.predict(X_unlabeled_tfidf)

for idx, text in enumerate(df_unlabeled['text']):
    if detect_suicide_risk(text):
        preds[idx] = 2

df_unlabeled['pseudo_label'] = preds
df_unlabeled['confidence'] = confidences

print(df_unlabeled[['text', 'pseudo_label', 'confidence']].head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


              precision    recall  f1-score   support

         0.0     0.8604    0.8976    0.8786       254
         1.0     0.9187    0.8898    0.9040       254
         2.0     0.9681    0.9567    0.9624       254

    accuracy                         0.9147       762
   macro avg     0.9157    0.9147    0.9150       762
weighted avg     0.9157    0.9147    0.9150       762

                                                  text  pseudo_label  \
173  Has anyone been hospitalised in a (mbu) mother...           0.0   
175  I’m 4 months PP and I’m still feeling this way...           2.0   
176  Hey all! A few months ago I had posted here as...           2.0   
178  Few weeks ago after DCF was investigating my f...           1.0   
179  I don’t know where I can vent so posting this ...           2.0   

     confidence  
173    0.577856  
175    0.709147  
176    0.781486  
178    0.493028  
179    0.536600  


In [12]:
high_conf_df = df_unlabeled[df_unlabeled['confidence'] >= 0.8]
low_conf_df = df_unlabeled[df_unlabeled['confidence'] <= 0.5]
print(f"\nNumber of pseudo-labeled samples with confidence greater than 0.8: {len(high_conf_df)} / {len(df_unlabeled)}")
print(f"\nNumber of pseudo-labeled samples with confidence less than or equal to 0.5: {len(low_conf_df)} / {len(df_unlabeled)}")


Number of pseudo-labeled samples with confidence greater than 0.8: 60 / 516

Number of pseudo-labeled samples with confidence less than or equal to 0.5: 100 / 516


In [15]:
high_conf_df["pseudo_label"].value_counts()

,count
pseudo_label,
0.0,44
2.0,13
1.0,3


In [16]:
low_conf_df["pseudo_label"].value_counts()

,count
pseudo_label,
2.0,38
1.0,36
0.0,26


In [17]:
high_conf_df.to_csv('high_conf_df_v2.csv', index=True)

In [18]:
low_conf_df.to_csv('low_conf_df_v2.csv', index=True)

In [20]:
df.to_csv('full_dataset_v2_170labels plus human revise_first.csv', index=True)